In [7]:
import cv2
import pytesseract
import time
import re
import speech_recognition as sr
import pyttsx3
import threading
import torch

# Configure Tesseract OCR (Update path if needed)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Initialize text-to-speech engine
engine = pyttsx3.init()
recognizer = sr.Recognizer()
stop_flag = False  # Flag to control thread execution
triggered = False  # Flag to check if "read" command was spoken

# Open webcam first to show the reading area
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)  # Optimize for smoother video
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

def text_to_speech(text):
    """Convert text to speech using pyttsx3."""
    engine.say(text)
    engine.runAndWait()

def is_valid_text(text):
    """Ensures detected text is meaningful and not just single letters or artifacts."""
    text = text.strip()
    if len(text) < 2 or re.fullmatch(r"[^A-Za-z]+", text):  # Reject short words & symbols
        return False
    return True

def listen_for_trigger():
    """Listen for the trigger word 'read' to activate OCR and TTS."""
    global stop_flag, triggered
    while not stop_flag:
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source)
            print("Listening for trigger word 'read'...")
            try:
                audio = recognizer.listen(source, timeout=100)
                spoken_text = recognizer.recognize_google(audio).lower()
                if "please" in spoken_text:
                    triggered = True
            except sr.UnknownValueError:
                pass
            except sr.RequestError:
                print("Speech recognition service unavailable")

def process_frame(frame):
    """Process a video frame for OCR detection using GPU if available."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Use GPU acceleration if available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        tensor_frame = torch.tensor(gray, device=device)
        gray = tensor_frame.cpu().numpy()
    
    detected_text = pytesseract.image_to_string(gray)
    valid_words = [word for word in detected_text.split() if is_valid_text(word)]
    return " ".join(valid_words)

print("Press 'q' to quit.")
trigger_thread = threading.Thread(target=listen_for_trigger)
trigger_thread.start()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture video. Exiting...")
        break
    
    # Always show the red box to indicate where it's looking
    h, w, _ = frame.shape
    cv2.rectangle(frame, (50, 50), (w - 50, h - 50), (0, 0, 255), 3)
    
    # Display video feed with red box
    cv2.imshow("Live OCR Feed - Say 'Read' to Capture", frame)
    
    # If trigger word is spoken, process the frame and read text
    if triggered:
        print("Trigger word detected. Capturing text...")
        full_text = process_frame(frame)
        if full_text:
            print(f"Detected Text: {full_text}")
            text_to_speech(full_text)
            time.sleep(2)  # Prevent repeating too fast
        triggered = False  # Reset trigger after processing
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        stop_flag = True
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
trigger_thread.join()


Press 'q' to quit.
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Detected Text: ROBIN SHARMA Will Cry You Die?
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Detected Text: ROBIN” SHARMA ho Will Cry
Listening for trigger word 'read'...
Listening for trigger word 'read'...


Press 'q' to quit.
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Detected Text: ROBIN SHARMA Who Will Cry When You Die?
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Detected Text: Fan Tr Who Will Cry ‘When You Die? lessons from the Monk who sold his Ferrari PAULO COELHO Author of The Alchemis
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Detected Text: wir Who Will Cr When You Die? af art
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Detected Text: Retr On tre Quest to Rediscover Microsoft's Soul and Imagine Better Future for Everyone
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Exception in thread Thread-93 (listen_for_trigger):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\Reach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Reach\AppData\Local\Temp\ipykernel_49808\3388397855.py", line 48, in listen_for_trigger
  File "C:\Users\Reach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\speech_recognition\__init__.py", line 460, in listen
    for a in result:
  File "C:\Users\Reach\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\speech_recognition\__init__.py", line 490, in _listen
    raise WaitTimeoutError("listening timed out while waiting for phrase to start")
speech_recognition.exceptions.WaitTimeoutError: listening timed out while waiting for phrase to start